In [1]:
#pip install xgboost

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import resample
from xgboost import XGBRegressor

In [3]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)


In [4]:
data=data[data['TARGET_B']==1]
data=data.drop(['TARGET_B'],axis=1)

In [5]:
y = data['TARGET_D']
X = data.drop(['TARGET_D'], axis = 1)

In [6]:
numericalX = X.select_dtypes(np.number).reset_index().drop(['index'],axis=1)
categoricalX = X.select_dtypes(np.object)

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

from sklearn.preprocessing import StandardScaler
transformer=StandardScaler().fit(numericalX)
X_norm=transformer.transform(numericalX)
X_trans=pd.DataFrame(X_norm,columns=numericalX.columns)

X = pd.concat([X_trans, encoded_categorical], axis = 1)

In [16]:
X

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-0.062573,-0.007716,-0.752569,0.337483,0.672618,0.218883,-0.497485,0.702676,-0.338281,0.999240,-0.109897,-0.506811,0.770704,-0.083687,-0.125743,-0.132519,0.832172,-0.430784,-0.639071,-0.636254,0.738330,0.558134,-0.351628,-0.246058,-0.144050,-0.489006,-0.339940,-0.188451,-0.261725,-0.274998,0.581341,-0.230249,-0.060552,-0.378212,-0.247546,-0.101784,-0.477315,-0.008574,-0.046863,-0.310218,0.090524,0.287206,0.274200,0.376919,-0.712544,0.745195,0.380267,-0.484590,-0.243700,0.983128,0.433095,-0.976060,-0.785429,1.062169,-0.242344,-0.925850,1.023419,0.289124,-0.222288,-0.680523,-0.203680,-0.590116,0.009579,-0.513630,0.331551,0.341543,0.127266,-0.455715,-0.307655,-0.133440,1.155363,-0.059157,0.117030,0.173109,0.744725,-0.653825,-0.334710,-0.393090,-0.289764,-0.253374,1.093414,1.844622,-0.155638,-0.253587,-0.268639,1.488879,1.692840,0.835362,-0.829676,0.553959,-0.621780,-0.160947,0.534094,0.033617,0.042375,0.529598,-0.217038,0.303271,0.129507,-0.555516,0.297509,-0.226174,0.217798,0.021712,0.613135,0.479597,-0.084978,-0.411675,-0.313465,-0.254774,-0.559849,-0.598038,-0.104483,0.517258,0.785514,-0.392231,-0.522013,0.809000,0.902944,0.769713,-0.041212,-0.294748,-0.636718,-0.082598,-0.588512,-0.793637,-0.564575,-0.224019,-0.532421,1.916535,1.444092,1.045345,0.764738,0.503461,-0.542098,-0.527293,0.672162,0.542519,0.834237,0.758274,0.404727,-1.041561,-1.075399,0.552599,0.632284,0.676078,0.827322,1.349174,-0.033462,-0.034523,-0.855569,-1.033651,-0.410022,0.754771,0.545250,1.021354,1.289314,-0.110644,-0.257786,-0.923718,-0.656196,1.303278,-0.987381,0.749892,-0.702938,0.709618,-0.216743,0.632641,-0.021688,0.106378,-0.216667,-0.126753,-0.362107,-0.375445,0.168870,-0.199099,3.000123,0.077336,-0.427138,-0.802649,0.401754,0.472462,0.546204,0.440964,0.662997,0.509673,0.971923,0.931497,0.927075,-1.374786,-0.603936,0.761019,0.981876,1.541854,-0.308788,0.282710,0.520828,-0.317747,-0.039345,-0.536062,-1.071824,-0.522814,-0.590329,-0.905328,-0.549095,-0.269053,-0.828644,0.622812,0.237827,-0.295005,-0.109980,-0.798901,-0.024269,-0.554904,

In [7]:
from sklearn.decomposition import PCA

In [8]:
#pca = PCA(n_components=5)
#X_pca= pca.fit_transform(X)
#X_pca=pd.DataFrame(X_pca)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
X_train

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
4550,-0.064296,-0.035795,0.445752,0.713906,-0.373528,0.000451,0.887231,-0.549588,0.759403,0.300892,-0.304853,-0.762194,0.770704,-0.205202,-0.205085,-0.255843,-1.269437,-0.430784,1.863274,0.319390,-0.187067,0.610926,-0.351628,-0.246058,-0.431083,-0.413526,-0.339940,-0.188451,-0.261725,-0.274998,-0.236149,-0.230249,-0.060552,-0.470544,0.570792,-0.101784,-0.172631,-0.601775,-0.775062,-0.931142,-0.873316,-0.594810,-0.773552,1.044007,0.890896,-0.061894,-1.069837,-0.484590,1.410009,0.983128,-0.728557,-0.976060,-0.623983,-0.414658,0.530954,1.082291,0.060821,-0.394024,-0.796546,-0.457635,-0.336568,-0.515036,-0.843157,-0.634795,1.088861,1.156591,0.762024,0.061396,0.984672,-1.235508,-0.294658,-0.529406,1.130574,0.929703,1.066476,1.127158,-0.139597,-0.690310,-0.690651,-0.646577,-0.297231,-0.276154,-0.155638,0.174708,0.162432,1.072320,0.831163,0.929700,-0.927241,-0.084485,0.176439,1.004038,1.226670,0.778571,0.997490,1.532764,0.741001,-0.789917,-0.518429,0.370918,-0.700126,-0.653605,-0.861831,-0.477816,1.660071,0.553552,-0.562323,-0.411675,-0.427104,-0.254774,-0.318380,0.138716,1.302362,1.143992,0.785514,-0.343193,-0.659973,0.669583,0.338473,0.574114,0.746129,-0.294748,-0.555418,-0.549235,-0.588512,-0.399100,-0.423278,-0.621280,-0.532421,1.461601,1.158830,0.828637,0.722205,0.683281,-1.247169,-1.475635,0.485398,0.361837,0.228229,0.146968,-0.201710,-0.613426,-0.826491,0.233866,0.632284,1.502804,-0.334239,0.179089,-0.529044,-0.403201,-0.769008,-0.494493,0.371149,0.162292,1.492465,-0.296870,0.042878,-0.530590,-0.415320,0.201601,-0.656196,-0.148609,-0.877536,0.061128,-0.015738,-0.853353,0.734995,-0.407928,-0.229267,-0.151217,-0.216667,-0.126753,-0.362107,-0.745165,-0.099625,0.783814,1.105295,0.683627,2.003864,3.959990,-0.760478,0.398453,0.546204,0.221411,0.730401,0.142502,-0.204369,0.158352,0.473992,0.819144,-0.487370,-0.435337,-0.424454,-0.203961,-0.843644,-0.244771,0.520828,0.641635,0.130031,-0.170098,1.340118,0.367791,1.262131,-0.202233,-0.039038,-0.269053,1.084986,-0.367983,0.859915,0.596824,-0.109980,0.167241,0.403903,-0.2246

In [13]:
model1 = LinearRegression()
model2 = XGBRegressor()
model3 =  RandomForestRegressor()
from sklearn.metrics import mean_absolute_error

model_pipeline = [model1, model2, model3]
model_names = ['Linear Regression', 'XGB', 'Random Forest Regressor']
scores = {}
scores_mae={}
val_scores= {}
val_scores_mae= {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=10))
    model.fit(X_train, y_train)
    scores[model_names[i]] = mean_score
    prediction=model.predict(X_test)
    scores_mae[model_names[i]] = mean_absolute_error(y_test, prediction)
    val_scores[model_names[i]] = model.score(X_test,y_test)
    val_scores_mae[model_names[i]]= mean_absolute_error(y_test, prediction)
    i = i+1
print(scores)
print(val_scores)
print(scores_mae)
print(val_scores_mae)

{'Linear Regression': 0.41178956348257545, 'XGB': 0.4214414338712002, 'Random Forest Regressor': 0.5184036278240203}
{'Linear Regression': 0.28955295013439475, 'XGB': 0.25929762797456113, 'Random Forest Regressor': 0.4098513331803194}
{'Linear Regression': 5.779741245394262, 'XGB': 5.534201443133831, 'Random Forest Regressor': 4.813602167182663}
{'Linear Regression': 5.779741245394262, 'XGB': 5.534201443133831, 'Random Forest Regressor': 4.813602167182663}


In [ ]:
# when improving the imbalance, i think it might be better in this case to downsample the data, reducing data for those who don't give money.
#this way, we don't create "false donors", that might lead to wrong decisions for the buisiness

In [14]:
np.mean(data['TARGET_D'])

15.624344414619037

In [15]:
np.std(data['TARGET_D'])

12.443852192994646